In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import scipy
from sklearn.model_selection import train_test_split

In [57]:
df1=pd.ExcelFile(r'C:\Users\achauhan\Desktop\Radian\Data\Standard Upload_RedwoodFlow_20180619.xlsx')

In [58]:
df=df1.parse()

In [4]:
copy_df = df.select_dtypes(include=['object']).copy()
copy_df.info()
copy_df.head(10)
copy_df1=copy_df
copy_df1= copy_df1.fillna(copy_df1['Final Compliance Grade'].value_counts().index[0])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12308 entries, 0 to 12307
Columns: 221 entries, Loan Status to Final Compliance Grade
dtypes: object(221)
memory usage: 20.8+ MB


In [5]:
print(copy_df1.isnull().values.sum())

0


In [6]:
print(copy_df1['Final Compliance Grade'].value_counts())
print(copy_df1['Final Property Valuations Grade'].value_counts())

1     8084
2     3823
3D     183
3      182
3C      36
Name: Final Compliance Grade, dtype: int64
1     10704
3D     1205
2       293
3        56
3C       50
Name: Final Property Valuations Grade, dtype: int64


In [7]:
Note_Status=copy_df[['Note Status', 'Final_Overall_Grade']]

In [8]:
#sns.set()
#_ = plt.hist(Note_Status['Note Status'])
#_ = plt.xlabel('Final_Overall_Grade')
#_ = plt.ylabel('Number')
#plt.show()

In [9]:
Note_Status.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12308 entries, 0 to 12307
Data columns (total 2 columns):
Note Status            12308 non-null object
Final_Overall_Grade    12308 non-null object
dtypes: object(2)
memory usage: 192.4+ KB


In [10]:
print(Note_Status.groupby(Note_Status['Note Status']))

In [11]:
Note_Status.groupby('Note Status')['Final_Overall_Grade'].sum()

Note Status
(No Data)     3C2111113D33D1313C113D111121111132131111111111...
Incomplete                                                   12
Present       22112222233D3D122213D1113D3D21132213D121222211...
Name: Final_Overall_Grade, dtype: object

In [12]:
note_status_groupby=Note_Status.groupby('Note Status').count()
note_status_groupby.head()

,Final_Overall_Grade
Note Status,
(No Data),597
Incomplete,2
Present,11709


In [13]:
train_test_split=df.copy()
train, test = train_test_split(df, test_size=0.2)

TypeError: 'DataFrame' object is not callable

In [ ]:
'''def get_split(dataset, n_features):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 999, 999, 999, None
	features = list()
	while len(features) < n_features:
		index = randrange(len(dataset[0])-1)
		if index not in features:
			features.append(index)
	for index in features:
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}'''


In [ ]:
'''
# Random Forest Algorithm on Sonar Dataset
from random import seed
from random import randrange
from csv import reader
from math import sqrt

# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini

# Select the best split point for a dataset
def get_split(dataset, n_features):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 999, 999, 999, None
	features = list()
	while len(features) < n_features:
		index = randrange(len(dataset[0])-1)
		if index not in features:
			features.append(index)
	for index in features:
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Create a terminal node value
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, n_features, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left, n_features)
		split(node['left'], max_depth, min_size, n_features, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right, n_features)
		split(node['right'], max_depth, min_size, n_features, depth+1)

# Build a decision tree
def build_tree(train, max_depth, min_size, n_features):
	root = get_split(train, n_features)
	split(root, max_depth, min_size, n_features, 1)
	return root

# Make a prediction with a decision tree
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']

# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio):
	sample = list()
	n_sample = round(len(dataset) * ratio)
	while len(sample) < n_sample:
		index = randrange(len(dataset))
		sample.append(dataset[index])
	return sample

# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
	predictions = [predict(tree, row) for tree in trees]
	return max(set(predictions), key=predictions.count)

# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
	trees = list()
	for i in range(n_trees):
		sample = subsample(train, sample_size)
		tree = build_tree(sample, max_depth, min_size, n_features)
		trees.append(tree)
	predictions = [bagging_predict(trees, row) for row in test]
	return(predictions)

# Test the random forest algorithm
seed(2)
# load and prepare data
filename = 'sonar.all-data.csv'
dataset = load_csv(filename)
# convert string attributes to integers
for i in range(0, len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1.0
n_features = int(sqrt(len(dataset[0])-1))
for n_trees in [1, 5, 10]:
	scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
	print('Trees: %d' % n_trees)
	print('Scores: %s' % scores)
	print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

# Random Forest Algorithm on Sonar Dataset
from random import seed
from random import randrange
from csv import reader
from math import sqrt
 
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset
 
# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())
 
# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup
 
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores
 
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right
 
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini
 
# Select the best split point for a dataset
def get_split(dataset, n_features):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 999, 999, 999, None
	features = list()
	while len(features) < n_features:
		index = randrange(len(dataset[0])-1)
		if index not in features:
			features.append(index)
	for index in features:
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}
 
# Create a terminal node value
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)
 
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, n_features, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left, n_features)
		split(node['left'], max_depth, min_size, n_features, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right, n_features)
		split(node['right'], max_depth, min_size, n_features, depth+1)
 
# Build a decision tree
def build_tree(train, max_depth, min_size, n_features):
	root = get_split(train, n_features)
	split(root, max_depth, min_size, n_features, 1)
	return root
 
# Make a prediction with a decision tree
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']
 
# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio):
	sample = list()
	n_sample = round(len(dataset) * ratio)
	while len(sample) < n_sample:
		index = randrange(len(dataset))
		sample.append(dataset[index])
	return sample
 
# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
	predictions = [predict(tree, row) for tree in trees]
	return max(set(predictions), key=predictions.count)
 
# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
	trees = list()
	for i in range(n_trees):
		sample = subsample(train, sample_size)
		tree = build_tree(sample, max_depth, min_size, n_features)
		trees.append(tree)
	predictions = [bagging_predict(trees, row) for row in test]
	return(predictions)
 
# Test the random forest algorithm
seed(2)
# load and prepare data
filename = 'sonar.all-data.csv'
dataset = load_csv(filename)
# convert string attributes to integers
for i in range(0, len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1.0
n_features = int(sqrt(len(dataset[0])-1))
for n_trees in [1, 5, 10]:
	scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
	print('Trees: %d' % n_trees)
	print('Scores: %s' % scores)
	print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))'''

In [14]:
df.head()

,Loan Number,Loan Status,Sample Group,Seller Loan Number,Borrower 1 Last Name,Borrower 1 First Name,Borrower 2 Last Name,Borrower 2 First Name,Borrower 3 First Name,Borrower 3 Last Name,...,General Comments,Loan Level ATR/QM Status,Initial Overall Grade,Initial Credit Grade,Initial Property Valuation Grade,Initial Compliance Grade,Final_Overall_Grade,Final Credit Grade,Final Property Valuations Grade,Final Compliance Grade
0,406040973,Client Complete,ONSLOW 02.2017,5215013788,ZACHARIAS,TIMOTHY,(No Data),(No Data),(No Data),(No Data),...,QM Safe Harbor Reviewed to Choice QM Guidelin...,QM: Safe Harbor,(No Data),(No Data),(No Data),(No Data),2,2,1,1
1,406061368,Client Complete,SEMT 2017-4 INITIAL,2003323,GLADWIN,DAVID,GLADWIN,JENNIFER,(No Data),(No Data),...,Final CD 04/05/2017\nNote: building plans in ...,QM: Safe Harbor,3D,3C,3D,2,2,1,1,2
2,406062431,Client Complete,SEMT 2017-CH1,4116218430,Lasry,Helena,(No Data),(No Data),(No Data),(No Data),...,Final CD 1/17/17 * QM N/A* Choice QM guideline...,ATR/QM: Not Applicable,3,2,3D,3,1,1,1,1
3,406062432,Client Complete,SEMT 2017-CH1,4116200476,Lasry,Helena,(No Data),(No Data),(No Data),(No Data),...,Final CD 1/18/17\n\nATR/QM: N/A\n\nIncome for ...,ATR/QM: Not Applicable,3,3D,3D,3,1,1,1,1
4,406064286,Client Complete,SEMT 2017-4 INITIAL,30089636,SMITH,Kevin,Hittson,Suzanne,(No Data),(No Data),...,Final CD 02/14/2017\nSelect QM 30 Year FRM gui...,QM: Safe Harbor,(No Data),(No Data),(No Data),(No Data),2,1,1,2


In [15]:
df.describe()

,Loan Number,Address Zip,Combined Loan Amount,Buydown Period,Note P&I Mod,Total Cash Out,Appraised Value,Number of Units,HCLTV,Borrower 2 Total Income,...,DTI (Back),Total Monthly Expenses,Primary Residence Expense,Effective Amount (Assets Used),Liquid Assets,Total Cash Required From Borrower,Cash Reserves,Months PITI Reserves,DTI Qualifying (Front),DTI Qualifying (Back)
count,1.230800e+04,12308.000000,1.230800e+04,12308.00000,12308.000000,1.230800e+04,1.230800e+04,12308.000000,12308.000000,12308.000000,...,12308.000000,12308.000000,12308.000000,1.230800e+04,1.230800e+04,1.230800e+04,1.230800e+04,12308.000000,12308.000000,12308.000000
mean,4.077483e+08,69694.850747,7.045753e+05,0.00195,7.116393,3.615084e+04,1.125731e+06,1.029737,1.313885,3836.202482,...,32.221752,7189.603947,4631.544799,4.687335e+05,3.881979e+05,1.713839e+05,3.369912e+05,67.446375,22.861282,32.355368
std,6.607452e+05,30764.205761,3.147527e+05,0.21633,157.509340,1.360600e+05,5.801346e+05,0.232123,9.670245,7357.303304,...,11.331100,4837.324787,2365.203694,1.087176e+06,1.055879e+06,2.499956e+05,1.059172e+06,185.281974,10.730848,11.368981
min,4.060410e+08,1262.000000,0.000000e+00,0.00000,0.000000,0.000000e+00,6.600000e+02,1.000000,0.000000,-27752.960000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
25%,4.080025e+08,40207.000000,5.320000e+05,0.00000,0.000000,0.000000e+00,7.600000e+05,1.000000,0.000000,0.000000,...,26.790000,4705.362500,3446.932500,1.206740e+05,6.107809e+04,0.000000e+00,6.655857e+04,14.837500,16.230000,26.900000
50%,4.080070e+08,84032.000000,6.598000e+05,0.00000,0.000000,0.000000e+00,9.750000e+05,1.000000,0.000000,0.000000,...,34.785000,6284.585000,4351.600000,2.756390e+05,2.116367e+05,1.224565e+05,1.454085e+05,30.500000,23.040000,35.005000
75%,4.080113e+08,94122.000000,8.450000e+05,0.00000,0.000000,0.000000e+00,1.307000e+06,1.000000,0.000000,6499.392500,...,40.230000,8616.335000,5602.197500,5.310799e+05,4.315299e+05,2.263020e+05,3.258627e+05,67.412500,30.262500,40.390000
max,4.080181e+08,99901.000000,6.451250e+06,24.00000,5549.330000,2.679765e+06,1.220000e+07,4.000000,89.997000,192708.640000,...,162.770000,115192.400000,47533.990000,8.606371e+07,8.606371e+07,6.723997e+06,8.606349e+07,11512.950000,108.400000,162.770000


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12308 entries, 0 to 12307
Columns: 244 entries, Loan Number to Final Compliance Grade
dtypes: float64(19), int64(4), object(221)
memory usage: 22.9+ MB


In [31]:
#X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.20, random_state=42)
df_imp.head()

,Address City,Address State,Address Zip,Loan Type,Product Type,Amortization Type,Original Principal Balance,Note Date,Combined Loan Amount,Note Rate,...,Borrower 2 Total Income,Representative Credit Score,Total Monthly Income,Total Remaining Gross Monthly Income,Total Monthly Expenses,Primary Residence Expense,Total Assets,Qualification Method,Loan Doc Type,Final_Overall_Grade
0,CLARKLAKE,MI,49234,Conventional,Jumbo 30yr Fixed,Fixed,1265000.00,01/20/2017,1265000.0,4.8750,...,0.00,709,35450.08,23633.32,11816.76,9054.76,496713.06,Fully Amortized,Full Doc,2
1,ROGERS,MN,55374,Conventional,Jumbo 30yr Fixed,Fixed,815750.00,04/06/2017,815750.0,4.0000,...,3863.61,803,24172.18,16225.19,7946.99,5297.33,476099.06,Fully Amortized,Full Doc,2
2,Brooklyn,NY,11215,Conventional,Jumbo 30yr Fixed,Fixed,800000.00,01/19/2017,800000.0,5.3750,...,0.00,769,12807.25,7411.64,5395.61,0.00,60883.76,Fully Amortized,Full Doc,1
3,Brooklyn,NY,11215,Conventional,Jumbo 30yr Fixed,Fixed,800000.00,01/19/2017,800000.0,5.3750,...,0.00,769,12807.25,7411.64,5395.61,0.00,60883.76,Fully Amortized,Full Doc,1
4,Rockwall,TX,75087,Conventional,Jumbo 30yr Fixed,Fixed,650000.00,02/14/2017,650000.0,4.5000,...,9629.62,705,20129.62,12352.53,7777.09,5007.09,558574.23,Fully Amortized,Full Doc,2


In [59]:
df_imp=df[['Address City','Address State','Address Zip','Loan Type',
           'Product Type', 'Amortization Type', 'Original Principal Balance',
           'Note Date', 'Combined Loan Amount', 'Note Rate', 'Note P&I', 'Note Term',
           'Interest Only Period','Lender Name', 'Origination Channel', 'Loan Modified',
           'Occupancy','Loan Purpose', 'Cash To Borrower At Close','Property Type',
           'Year Built', 'Occupancy Status', 'LTV','Title Status', 'Other Liens',
           'Property Tax Amount','Borrower 1 Age', 'Borrower 1 Years On Job',
           'Borrower 1 Years In Field','Borrower 1 Self Employed', 'Borrower 1 First Time Home Buyer',
           'Borrower 1 Equifax Credit Score', 'Borrower 1 Experian Credit Score',
           'Borrower 1 Transunion Credit Score', 'Borrower 1 Total Income',
           'Borrower 2 Equifax Credit Score', 'Borrower 2 Experian Credit Score',
           'Borrower 2 Transunion Credit Score', 'Borrower 2 Total Income',
           'Representative Credit Score','Total Monthly Income',
           'Total Remaining Gross Monthly Income','Total Monthly Expenses',
           'Primary Residence Expense', 'Total Assets','Qualification Method',
          'Loan Doc Type','Final_Overall_Grade']]

In [60]:
df_imp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12308 entries, 0 to 12307
Data columns (total 48 columns):
Address City                            12308 non-null object
Address State                           12308 non-null object
Address Zip                             12308 non-null int64
Loan Type                               12308 non-null object
Product Type                            12308 non-null object
Amortization Type                       12308 non-null object
Original Principal Balance              12308 non-null object
Note Date                               12308 non-null object
Combined Loan Amount                    12308 non-null float64
Note Rate                               12308 non-null object
Note P&I                                12308 non-null object
Note Term                               12308 non-null object
Interest Only Period                    12308 non-null object
Lender Name                             12308 non-null object
Origination Channel    

In [61]:
df_imp.info

<bound method DataFrame.info of               Address City Address State  Address Zip     Loan Type  \
0                CLARKLAKE            MI        49234  Conventional   
1                   ROGERS            MN        55374  Conventional   
2                 Brooklyn            NY        11215  Conventional   
3                 Brooklyn            NY        11215  Conventional   
4                 Rockwall            TX        75087  Conventional   
5                 Meridian            ID        83646  Conventional   
6                  Durango            CO        81301  Conventional   
7                Lancaster            VA        22503  Conventional   
8                   Dallas            TX        75230  Conventional   
9               CLYDE HILL            WA        98004  Conventional   
10                   Tryon            NC        28782  Conventional   
11                  Eugene            OR        97402  Conventional   
12                Coronado            CA     

In [30]:
df_imp.describe()

,Address Zip,Combined Loan Amount,Borrower 2 Total Income,Total Monthly Income,Total Remaining Gross Monthly Income,Total Monthly Expenses,Primary Residence Expense
count,12308.000000,1.230800e+04,12308.000000,12308.000000,12308.000000,12308.000000,12308.000000
mean,69694.850747,7.045753e+05,3836.202482,23604.255375,16414.651428,7189.603947,4631.544799
std,30764.205761,3.147527e+05,7357.303304,25338.589974,22016.024731,4837.324787,2365.203694
min,1262.000000,0.000000e+00,-27752.960000,-2404.290000,-9041.890000,0.000000,0.000000
25%,40207.000000,5.320000e+05,0.000000,13727.930000,8495.685000,4705.362500,3446.932500
50%,84032.000000,6.598000e+05,0.000000,19005.375000,12344.070000,6284.585000,4351.600000
75%,94122.000000,8.450000e+05,6499.392500,26908.785000,18329.195000,8616.335000,5602.197500
max,99901.000000,6.451250e+06,192708.640000,966361.000000,857896.730000,115192.400000,47533.990000


In [62]:
df_imp.groupby(df_imp['Address State']='No Data').count()

SyntaxError: keyword can't be an expression (<ipython-input-62-0d502d6adcfa>, line 1)

In [63]:
df_imp['Address State'].value_counts()

CA    3921
WA    1125
TX    1045
CO     699
FL     531
MA     465
AZ     450
IL     362
GA     340
OR     324
VA     294
NY     254
NC     196
MN     193
NJ     172
SC     162
MD     162
UT     151
TN     126
NV     119
MO     117
WI     103
PA     100
MT      92
MI      79
AL      59
ID      59
DC      51
LA      49
HI      49
NM      44
IN      39
DE      38
CT      37
AR      34
SD      30
OH      30
WY      29
KY      27
NH      26
KS      24
OK      24
ME      18
VT      16
IA      13
RI      12
NE       8
MS       5
ND       3
AK       2
Name: Address State, dtype: int64

In [65]:
imp_city=df_imp['Address City'].value_counts()

In [70]:
imp_product=df_imp['Product Type'].value_counts()

In [71]:
imp_product

Jumbo 30yr Fixed             9543
(No Data)                    1093
Jumbo 7/1 ARM                 522
30 Yr Fixed                   432
Jumbo 15yr Fixed              226
Jumbo 5/1 ARM                 182
7/1 ARM                        77
Jumbo 10/1 ARM                 40
Jumbo 7/1 ARM IO               35
Fixed                          24
5/1 ARM                        21
20 Yr Fixed                    20
Jumbo 5/1 ARM IO               14
15 Yr Fixed                    13
7/1 I/O ARM                    12
Jumbo 30yr Fixed IO            11
10/1 ARM                        7
Jumbo 10/1 ARM IO               7
5/1 I/O ARM                     5
Jumbo Plus 7/1 ARM              5
30 yr Fixed - 10 yr IO          4
25 Yr Fixed                     4
10/1 I/O ARM                    2
Jumbo Plus 30yr Fixed           2
30/15 Interest Only             1
10/20 Interest Only             1
10/1 ARM 1 Year Libor I/O       1
10 Yr Fixed                     1
Fixed Rate                      1
30 Yr Fixed In

In [39]:
df_imp.dtypes

Address City                             object
Address State                            object
Address Zip                               int64
Loan Type                                object
Product Type                             object
Amortization Type                        object
Original Principal Balance               object
Note Date                                object
Combined Loan Amount                    float64
Note Rate                                object
Note P&I                                 object
Note Term                                object
Interest Only Period                     object
Lender Name                              object
Origination Channel                      object
Loan Modified                            object
Occupancy                                object
Loan Purpose                             object
Cash To Borrower At Close                object
Property Type                            object
Year Built                              

In [43]:
df_imp['Borrower 2 Equifax Credit Score'].value_counts()

(No Data)    5106
799           154
791           151
802           146
801           143
800           135
806           133
792           129
795           127
788           125
789           122
790           121
794           118
784           117
785           116
797           115
782           115
798           110
796           110
803           109
780           108
787           103
809           102
777           101
793           101
804            99
778            98
781            92
786            88
808            88
             ... 
659             3
667             3
670             3
820             2
673             2
658             2
655             2
676             2
652             2
666             2
672             2
650             2
653             2
642             2
654             2
669             1
649             1
824             1
629             1
641             1
651             1
621             1
823             1
819             1
822       

In [84]:
df_imp['Origination Channel'].value_counts()

Retail                                               9394
Broker                                               1695
(No Data)                                            1102
Correspondent Flow with delegated underwriting         68
Correspondent Flow without delegated underwriting      24
Unknown                                                16
Correspondent Bulk                                      8
Unavailable                                             1
Name: Origination Channel, dtype: int64

In [85]:
df_imp['Loan Modified'].value_counts()

No                11185
(No Data)          1093
Yes                  29
Not Applicable        1
Name: Loan Modified, dtype: int64

In [86]:
df_imp['Occupancy'].value_counts()

Primary        10544
Second Home      600
(No Data)        597
Investor         567
Name: Occupancy, dtype: int64

In [87]:
df_imp['Loan Purpose'].value_counts()

Purchase              7645
Rate and Term Refi    2034
Cashout Refi          1860
(No Data)              597
Construction/Perm      168
Home Improvement         2
Streamline               2
Name: Loan Purpose, dtype: int64

In [90]:
df_imp['Cash To Borrower At Close'].value_counts()

(No Data)         9702
0.00                35
2000.00             17
Not Applicable       3
9.00                 2
3343.90              2
5000.00              2
500000.00            2
1931.20              2
Missing              2
25.00                2
750000.00            2
20004.38             1
1791.33              1
90559.63             1
1673.45              1
102664.84            1
73924.05             1
114832.96            1
1047.40              1
708712.77            1
554.77               1
82813.08             1
51592.30             1
7545.21              1
121322.73            1
53766.59             1
139974.82            1
33914.06             1
912.53               1
                  ... 
276935.91            1
88075.66             1
14834.11             1
44265.54             1
85.58                1
1162.51              1
2137.45              1
1063.89              1
10535.81             1
114835.05            1
1883.47              1
356933.40            1
22225.09   

In [91]:
df_imp['Property Type'].value_counts()

SFR                      6966
Detached PUD             4004
Low Rise Condo (1-4)      571
High Rise Condo (9+)      175
2 Family                  166
Mid Rise Condo  (5-8)     113
SFR-Attached               86
Attached PUD               76
3 Family                   46
Site Condo                 41
4 Family                   36
Cooperative                15
Rowhouse                   10
Condotel                    3
Name: Property Type, dtype: int64

In [92]:
df_imp['Year Built'].value_counts()

2017           1334
(No Data)       541
2016            449
2018            366
2006            324
2005            320
2004            277
2007            270
2003            234
2000            222
2001            209
2015            204
1999            197
2002            192
2014            176
2008            168
2013            158
1990            156
1998            152
1997            148
1995            140
1989            132
1994            126
1988            125
1978            125
1987            119
1993            110
2012            109
1996            107
1979            107
               ... 
1812              2
1878              2
1855              2
1820              2
Missing           2
1760              1
2207              1
1765              1
1868              1
1825              1
1834              1
1811              1
1800              1
1865              1
Unavailable       1
1842              1
1882              1
1849              1
1843              1


In [94]:
df_imp['LTV'].value_counts()

80.00        3596
75.00         757
(No Data)     597
90.00         552
70.00         527
60.00         242
65.00         238
85.00         144
50.00          62
66.67          37
62.50          25
55.00          23
58.82          21
61.54          20
79.99          16
76.92          15
69.23          15
63.64          15
71.43          15
55.56          15
57.14          15
58.33          13
68.00          12
45.45          12
40.00          11
69.93          11
73.33          10
64.29          10
89.90          10
68.75          10
             ... 
66.82           1
57.61           1
57.10           1
55.27           1
56.08           1
63.05           1
40.97           1
65.63           1
53.78           1
40.35           1
63.32           1
61.41           1
56.51           1
88.75           1
25.53           1
52.10           1
73.34           1
49.02           1
60.47           1
77.33           1
81.36           1
25.13           1
48.12           1
64.14           1
86.47     

In [97]:
df_imp['Other Liens'].value_counts()

No           11417
(No Data)      597
Yes            294
Name: Other Liens, dtype: int64

In [99]:
df_imp['Borrower 1 Age'].value_counts()

(No Data)    1234
48            422
39            416
40            416
44            384
37            381
38            379
46            376
47            376
42            372
41            371
43            367
45            350
36            344
34            340
35            328
33            300
50            299
54            296
53            293
49            287
52            284
51            280
55            257
32            236
56            235
58            205
57            199
31            198
60            182
             ... 
69             76
68             70
28             69
70             67
71             54
27             45
72             44
73             40
74             37
75             37
26             28
76             24
77             22
25             19
78             15
79             12
83             12
82              6
80              6
81              6
24              6
22              3
21              2
23              2
88        

In [100]:
df_imp['Borrower 1 Years On Job'].value_counts()

(No Data)    597
0.00         486
10.00        289
0.08         264
3.00         227
2.00         227
5.00         217
6.00         202
4.00         197
8.00         173
12.00        162
15.00        159
7.00         157
11.00        147
20.00        136
1.00         135
9.00         129
0.17         129
13.00        124
17.00        120
0.50         112
0.25         110
14.00        108
16.00        103
0.33         100
0.75          97
2.50          91
18.00         89
0.58          88
1.25          85
            ... 
13.01          1
15.30          1
38.58          1
18.70          1
23.67          1
0.06           1
12.11          1
5.30           1
3.40           1
0.07           1
35.08          1
46.33          1
37.17          1
27.10          1
29.42          1
6.40           1
38.33          1
30.83          1
4.90           1
2.11           1
39.83          1
28.70          1
0.82           1
39.17          1
37.08          1
38.17          1
7.60           1
1.30          

In [101]:
df_imp['Borrower 1 Years In Field'].value_counts()

10.00        1009
20.00         859
15.00         688
(No Data)     597
0.00          581
12.00         460
5.00          457
25.00         420
30.00         410
17.00         349
11.00         328
8.00          324
6.00          317
7.00          312
18.00         291
13.00         290
16.00         289
9.00          271
4.00          263
3.00          248
14.00         243
2.00          216
22.00         197
19.00         185
21.00         184
24.00         149
23.00         145
35.00         134
27.00         129
26.00         121
             ... 
22.25           1
23.33           1
1.33            1
35.75           1
49.00           1
25.83           1
31.25           1
31.42           1
24.17           1
8.75            1
16.42           1
60.00           1
54.00           1
26.50           1
30.67           1
31.33           1
21.92           1
37.42           1
1.08            1
39.17           1
26.58           1
1.83            1
4.10            1
2.92            1
1.17      

In [102]:
df_imp['Borrower 2 Experian Credit Score'].value_counts()

(No Data)      5111
790             155
776             135
793             134
779             127
804             126
787             126
796             123
797             121
803             118
801             115
798             107
788             107
782             104
773              99
783              98
792              97
770              94
791              92
766              91
786              90
809              86
800              86
807              85
794              84
785              83
763              83
784              81
808              81
781              81
               ... 
663               3
694               3
671               3
651               3
668               3
658               3
662               3
656               2
672               2
640               2
653               2
655               2
660               2
Unavailable       2
836               2
844               2
665               2
629               1
652               1


In [104]:
df_imp['Borrower 2 Total Income'].value_counts()

 0.00        7297
 5000.00       28
 12500.00      26
 10000.00      21
 7500.00       17
 0.01          17
 8333.33       14
 6500.00       14
 11250.00      13
 2500.00       12
 10416.67      11
 8333.34       11
 4333.33       10
 6250.00       10
 5416.67       10
 9166.67       10
 8750.00        8
 15000.00       8
 11666.67       8
 7083.34        7
 10416.66       7
 5833.34        7
 6666.66        7
 7083.33        7
 6000.00        7
 13333.34       6
 12083.33       6
 3466.67        6
 9000.00        6
 9999.99        6
             ... 
 2234.07        1
 12177.46       1
 15125.37       1
 20529.05       1
 13086.12       1
 141.46         1
 1900.32        1
 11689.00       1
 9583.29        1
 11524.15       1
 36653.58       1
 8989.00        1
 4100.92        1
 12767.73       1
 6166.68        1
 12399.79       1
 8616.25        1
-674.00         1
 6484.00        1
 22550.30       1
 13499.98       1
 9416.58        1
 3341.00        1
 4483.00        1
 17602.50 

In [105]:
df_imp['Representative Credit Score'].value_counts()

(No Data)         597
776               207
787               194
791               192
790               187
779               186
773               185
782               185
777               176
788               174
801               170
778               168
770               165
781               163
797               161
792               160
783               160
784               159
786               159
798               158
763               157
774               156
789               154
796               154
766               152
780               151
775               148
785               147
800               147
771               145
                 ... 
695                12
819                12
670                11
673                11
674                11
675                10
668                10
671                10
669                 9
679                 8
664                 8
820                 8
665                 7
678                 6
823       

In [106]:
df_imp['Total Monthly Income'].value_counts()

0.00        598
12500.00     41
25000.00     34
15000.00     31
20000.00     31
18750.00     26
16666.67     19
17500.00     18
20833.34     17
20833.33     17
13333.33     17
14166.67     16
10000.00     16
16250.00     15
13333.34     15
13750.00     15
21666.67     14
14583.33     14
29166.67     14
16666.66     11
14166.66     11
20833.00     10
15833.33     10
22500.00     10
13000.00      9
14583.34      9
11250.00      9
19166.67      9
31250.00      9
26250.00      8
           ... 
25307.30      1
23642.22      1
23401.39      1
10099.28      1
13625.00      1
8705.83       1
29407.33      1
11093.33      1
23376.00      1
27127.46      1
8250.37       1
34891.46      1
31521.17      1
18326.29      1
23575.00      1
13684.67      1
16143.00      1
13833.59      1
11505.30      1
9590.41       1
31994.06      1
77899.33      1
17906.99      1
11944.07      1
16448.51      1
15931.60      1
17978.00      1
14256.66      1
15857.05      1
21826.73      1
Name: Total Monthly Inco

In [107]:
df_imp['Total Remaining Gross Monthly Income'].value_counts()

0.00        597
9947.60       3
12362.51      2
10679.65      2
15322.87      2
11187.14      2
10974.50      2
4651.57       2
16125.18      2
9887.97       2
11613.90      2
15373.19      2
6539.54       2
18282.84      2
17319.52      2
19640.31      2
24663.64      2
10337.02      2
5257.79       2
10594.50      2
14175.22      2
7411.64       2
9493.95       2
8601.97       2
10647.69      2
10982.11      2
9707.18       2
14926.69      2
13350.12      2
10345.20      2
           ... 
13001.52      1
20096.13      1
6649.90       1
12346.64      1
8389.08       1
11729.68      1
11135.53      1
8061.30       1
13723.26      1
12472.47      1
12536.75      1
14490.76      1
13527.28      1
20610.19      1
20435.11      1
17653.29      1
6318.65       1
8111.72       1
9871.97       1
8454.15       1
14032.14      1
7129.69       1
29835.34      1
14716.86      1
7054.33       1
8308.41       1
17718.27      1
24876.19      1
9231.98       1
5209.47       1
Name: Total Remaining Gr

In [108]:
df_imp['Total Monthly Expenses'].value_counts()

0.00        597
3029.71       4
7234.94       3
19573.70      2
7161.77       2
7135.26       2
6628.60       2
6790.74       2
4983.44       2
9778.89       2
6959.55       2
9751.87       2
7156.48       2
6783.08       2
8909.60       2
4226.48       2
15790.32      2
5248.92       2
7174.42       2
6088.82       2
10995.88      2
8147.23       2
9364.91       2
4786.94       2
3602.97       2
4812.68       2
5147.62       2
5395.61       2
5252.95       2
7107.82       2
           ... 
7651.04       1
5335.31       1
3252.91       1
7413.84       1
5559.27       1
4201.06       1
8436.77       1
5581.19       1
4751.31       1
6056.71       1
6765.65       1
7253.00       1
11052.86      1
4115.37       1
3113.63       1
4074.15       1
5278.76       1
6087.85       1
7312.54       1
6524.71       1
5892.42       1
13113.39      1
7017.57       1
5037.55       1
6720.44       1
6679.97       1
4908.60       1
6782.71       1
3428.49       1
6833.74       1
Name: Total Monthly Expe

In [109]:
df_imp['Primary Residence Expense'].value_counts()

0.00       652
2334.00      5
2813.71      5
2778.00      3
3500.00      3
4777.45      3
2053.00      3
5275.54      2
7826.91      2
5968.48      2
5187.00      2
4943.00      2
3901.63      2
5087.20      2
3252.86      2
3644.64      2
3485.55      2
4912.60      2
3504.59      2
3153.61      2
6349.52      2
4054.23      2
4706.09      2
4291.00      2
4537.18      2
4866.74      2
5000.00      2
5956.58      2
3304.47      2
6027.78      2
          ... 
4630.43      1
4888.36      1
5490.42      1
4883.40      1
7974.10      1
2972.17      1
4162.24      1
4758.81      1
5480.32      1
7188.25      1
3723.84      1
3748.74      1
4613.92      1
2787.90      1
7566.81      1
5078.95      1
1546.69      1
2733.00      1
5311.64      1
4375.44      1
6091.31      1
5652.06      1
5585.82      1
4534.81      1
2866.38      1
6214.10      1
6909.30      1
5339.24      1
3569.27      1
3129.63      1
Name: Primary Residence Expense, Length: 11499, dtype: int64

In [112]:
df_imp['Loan Doc Type'].value_counts()

Full Doc     10981
(No Data)     1143
Full           184
Name: Loan Doc Type, dtype: int64

In [116]:
df_imp['Loan Type'].value_counts()

Conventional    12308
Name: Loan Type, dtype: int64